# Proyecto Planes de Megaline - Clasificación

In [1]:
#importar librerias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Cargar los datos
df = pd.read_csv('/datasets/users_behavior.csv')

# Examinar los datos
print(df.head())
print(df.info())
print(df['is_ultra'].value_counts())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
0    2229
1     985
Name: is_ultra, dtype: int64


Se puede observar que no hay valores nulos, y la mayor parte de usuarios tienen como plan actual el Smart (0).

In [3]:
# Segmentar los datos

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=54321)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=54321)

# Extraer características y objetivos
features_train = train_df.drop('is_ultra', axis=1)
target_train = train_df['is_ultra']

features_valid = valid_df.drop('is_ultra', axis=1)
target_valid = valid_df['is_ultra']

features_test = test_df.drop('is_ultra', axis=1)
target_test = test_df['is_ultra']

In [4]:
# Modelo 1: Árbol de decisión
# Ajustar hiperparámetros: max_depth, min_samples_split, min_samples_leaf
best_tree_accuracy = 0
best_tree_params = {}

for max_depth in range(1, 11):  # Probar profundidades de 1 a 10
    for min_samples_split in [2, 5, 10]:  # Probar diferentes divisiones mínimas
        for min_samples_leaf in [1, 2, 4]:  # Probar diferentes hojas mínimas
            tree_model = DecisionTreeClassifier(
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=54321
            )
            tree_model.fit(features_train, target_train)
            tree_predictions = tree_model.predict(features_valid)
            tree_accuracy = accuracy_score(target_valid, tree_predictions)
            
            # Guardar la mejor configuración
            if tree_accuracy > best_tree_accuracy:
                best_tree_accuracy = tree_accuracy
                best_tree_params = {
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf
                }

print(f"Mejor exactitud del Árbol de Decisión: {best_tree_accuracy:.2f}")
print(f"Mejores hiperparámetros del Árbol de Decisión: {best_tree_params}")

# Modelo 2: Bosque aleatorio
# Ajustar hiperparámetros: n_estimators, max_depth, min_samples_split
best_forest_accuracy = 0
best_forest_params = {}

for n_estimators in [50, 100, 150]:  # Probar diferentes números de árboles
    for max_depth in range(5, 16, 5):  # Probar profundidades de 5, 10, 15
        for min_samples_split in [2, 5, 10]:  # Probar diferentes divisiones mínimas
            forest_model = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                random_state=54321
            )
            forest_model.fit(features_train, target_train)
            forest_predictions = forest_model.predict(features_valid)
            forest_accuracy = accuracy_score(target_valid, forest_predictions)
            
            # Guardar la mejor configuración
            if forest_accuracy > best_forest_accuracy:
                best_forest_accuracy = forest_accuracy
                best_forest_params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split
                }

print(f"Mejor exactitud del Bosque Aleatorio: {best_forest_accuracy:.2f}")
print(f"Mejores hiperparámetros del Bosque Aleatorio: {best_forest_params}")

# Modelo 3: Regresión logística
# Ajustar hiperparámetros: C, solver, max_iter
best_logistic_accuracy = 0
best_logistic_params = {}

for C in [0.01, 0.1, 1, 10, 100]:  # Probar diferentes valores de regularización
    for solver in ['liblinear', 'lbfgs']:  # Probar diferentes algoritmos de optimización
        for max_iter in [100, 200, 300]:  # Probar diferentes iteraciones máximas
            logistic_model = LogisticRegression(
                C=C,
                solver=solver,
                max_iter=max_iter,
                random_state=54321
            )
            logistic_model.fit(features_train, target_train)
            logistic_predictions = logistic_model.predict(features_valid)
            logistic_accuracy = accuracy_score(target_valid, logistic_predictions)
            
            # Guardar la mejor configuración
            if logistic_accuracy > best_logistic_accuracy:
                best_logistic_accuracy = logistic_accuracy
                best_logistic_params = {
                    'C': C,
                    'solver': solver,
                    'max_iter': max_iter
                }

print(f"Mejor exactitud de la Regresión Logística: {best_logistic_accuracy:.2f}")
print(f"Mejores hiperparámetros de la Regresión Logística: {best_logistic_params}")

Mejor exactitud del Árbol de Decisión: 0.78
Mejores hiperparámetros del Árbol de Decisión: {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1}
Mejor exactitud del Bosque Aleatorio: 0.80
Mejores hiperparámetros del Bosque Aleatorio: {'n_estimators': 100, 'max_depth': 10, 'min_samples_split': 2}
Mejor exactitud de la Regresión Logística: 0.68
Mejores hiperparámetros de la Regresión Logística: {'C': 0.1, 'solver': 'liblinear', 'max_iter': 100}


El Bosque Aleatorio es el mejor modelo, ya que tiene la mayor exactitud (80%). 

Este modelo supera el umbral de exactitud requerido (0.75), lo que indica que es adecuado para recomendar los planes Smart o Ultra a los clientes.

In [5]:
# Seleccionar el mejor modelo (Bosque Aleatorio)
best_model = RandomForestClassifier(random_state=54321)
best_model.fit(features_train, target_train)

# Predecir en el conjunto de prueba
test_predictions = best_model.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)
print("Exactitud del mejor modelo en el conjunto de prueba:", test_accuracy)

Exactitud del mejor modelo en el conjunto de prueba: 0.8102643856920684


In [6]:
#Prueba de cordura
# Modelo de referencia: Para predicir la clase mayoritaria (Smart - 0)
baseline_predictions = [0] * len(target_test)
baseline_accuracy = accuracy_score(target_test, baseline_predictions)
print("Exactitud del modelo de referencia:", baseline_accuracy)

# Comparar con el mejor modelo
if test_accuracy > baseline_accuracy:
    print("El modelo supera la prueba de cordura.")
else:
    print("El modelo no supera la prueba de cordura.")

Exactitud del modelo de referencia: 0.7247278382581649
El modelo supera la prueba de cordura.


El modelo de Bosque Aleatorio supera la prueba de cordura, ya que su exactitud (81,03%) es mayor que la del modelo de referencia (72.47%). El modelo de referencia o baseline en este caso es un modelo que siempre predice la clase mayoritaria (en este caso el plan Smart - 0). 

CONCLUSIÓN:

El Bosque Aleatorio es el mejor modelo, ya que tiene la mayor exactitud tanto en el conjunto de validación (78.23%) como en el conjunto de prueba (81.03%).

Este modelo supera el umbral de exactitud requerido (0.75), lo que indica que es adecuado para recomendar los planes Smart o Ultra a los clientes de Megaline basándose en su comportamiento mensual.